<a href="https://colab.research.google.com/github/randamzz/energy-contol-with-IoT-and-ML/blob/randa's_branche/prediction_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pt1. Nettoyage et Analyse des donnes**



In [61]:
import pandas as pd

# Lire le fichier CSV en spécifiant le séparateur comme étant un point-virgule
data = pd.read_csv("/content/consommation-quotidienne-brute.csv", sep=";")





,Date - Heure,Date,Heure,Consommation brute électricité (MW) - RTE,Statut - RTE,Consommation brute totale (MW)
0,2024-10-31T23:30:00+01:00,31/10/2024,23:30,47110,Consolidé,NaN
1,2024-10-31T23:00:00+01:00,31/10/2024,23:00,47753,Consolidé,80485.0
2,2024-10-31T22:30:00+01:00,31/10/2024,22:30,48552,Consolidé,NaN
3,2024-10-31T22:00:00+01:00,31/10/2024,22:00,47498,Consolidé,83039.0
4,2024-10-31T21:30:00+01:00,31/10/2024,21:30,46852,Consolidé,NaN


In [64]:

# Sélectionner les colonnes souhaitées
colonnes_a_selectionner = ['Date - Heure', 'Date', 'Heure', 'Consommation brute électricité (MW) - RTE' ,'Statut - RTE' ,'Consommation brute totale (MW)']
data2 = data[colonnes_a_selectionner]
data2.head()

,Date - Heure,Date,Heure,Consommation brute électricité (MW) - RTE,Statut - RTE,Consommation brute totale (MW)
0,2024-10-31T23:30:00+01:00,31/10/2024,23:30,47110,Consolidé,NaN
1,2024-10-31T23:00:00+01:00,31/10/2024,23:00,47753,Consolidé,80485.0
2,2024-10-31T22:30:00+01:00,31/10/2024,22:30,48552,Consolidé,NaN
3,2024-10-31T22:00:00+01:00,31/10/2024,22:00,47498,Consolidé,83039.0
4,2024-10-31T21:30:00+01:00,31/10/2024,21:30,46852,Consolidé,NaN


In [65]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225024 entries, 0 to 225023
Data columns (total 6 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Date - Heure                               225024 non-null  object 
 1   Date                                       225024 non-null  object 
 2   Heure                                      225024 non-null  object 
 3   Consommation brute électricité (MW) - RTE  225024 non-null  int64  
 4   Statut - RTE                               225024 non-null  object 
 5   Consommation brute totale (MW)             112508 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.3+ MB


In [69]:
data2['Statut - RTE'].unique()

array(['Consolidé', 'Définitif'], dtype=object)

In [66]:
data2.shape

(225024, 6)

In [68]:

data2['Date'] = pd.to_datetime(data2['Date'])
# Créer une colonne "Mois" pour regrouper par mois
data2['Mois'] = data2['Date'].dt.to_period('M')

<ipython-input-68-9f7121e6b2fc>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data2['Date'] = pd.to_datetime(data2['Date'])
<ipython-input-68-9f7121e6b2fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Date'] = pd.to_datetime(data2['Date'])
<ipython-input-68-9f7121e6b2fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Mois'] = data2['Date'].dt.to_period('M')


In [71]:

# Regrouper par mois et sommer les consommations
monthly_consumption = data2.groupby('Mois')['Consommation brute électricité (MW) - RTE'].sum().reset_index()

# Afficher les premières lignes de la consommation mensuelle
monthly_consumption.head()

,Mois,Consommation brute électricité (MW) - RTE
0,2012-01,101573595
1,2012-02,108363112
2,2012-03,85890312
3,2012-04,79898631
4,2012-05,70076935


In [77]:

# Définir une fonction pour calculer l'unité énergétique
def calculate_unit(year, season):
    base_rate = 0.000002  # DH par kWh
    # Augmenter l'unité de 5% tous les 3 ans
    years_since_2012 = year - 2012
    increase_rate = (years_since_2012 // 3) * 0.5  # Augmentation tous les 3 ans de 5%
    unit = base_rate * (1 + increase_rate)

    # Si la saison est été, augmenter encore un peu l'unité
    if season == 'Été':
        unit *= 1.5  # Augmentation de 10% pour l'été

    # Debugging: Affichage de l'unité calculée
   # print(f"Année: {year}, Saison: {season}, Unité: {unit}")
    return unit

In [78]:
# Ajouter une colonne "Année" et "Saison" dans le DataFrame des consommations mensuelles
monthly_consumption['Année'] = monthly_consumption['Mois'].dt.year
monthly_consumption['Saison'] = monthly_consumption['Mois'].dt.month.apply(lambda x: 'Hiver' if x in [12, 1, 2] else 'Été')

# Ajouter la colonne "Unité énergétique" en calculant pour chaque ligne
monthly_consumption['Unité énergétique'] = monthly_consumption.apply(
    lambda row: calculate_unit(row['Année'], row['Saison']), axis=1
)

# Afficher les premières lignes avec l'unité énergétique
monthly_consumption.head()

,Mois,Consommation brute électricité (MW) - RTE,Année,Saison,Unité énergétique,Coût énergétique (DH)
0,2012-01,101573595,2012,Hiver,0.000002,203.15
1,2012-02,108363112,2012,Hiver,0.000002,216.73
2,2012-03,85890312,2012,Été,0.000003,257.67
3,2012-04,79898631,2012,Été,0.000003,239.70
4,2012-05,70076935,2012,Été,0.000003,210.23


In [81]:

# Calculer le coût énergétique en DH
monthly_consumption['Coût énergétique (DH)'] = monthly_consumption['Consommation brute électricité (MW) - RTE'] * monthly_consumption['Unité énergétique']
# Arrondir à 2 décimales pour éviter les valeurs trop petites ou trop grandes
monthly_consumption['Coût énergétique (DH)'] = monthly_consumption['Coût énergétique (DH)'].round(2)

monthly_consumption = monthly_consumption.rename(columns={
    'Consommation brute électricité (MW) - RTE': 'Consommation électricité (MW)'
})
# Afficher les résultats finaux
monthly_consumption


,Mois,Consommation électricité (MW),Année,Saison,Unité énergétique,Coût énergétique (DH)
0,2012-01,101573595,2012,Hiver,0.000002,203.15
1,2012-02,108363112,2012,Hiver,0.000002,216.73
2,2012-03,85890312,2012,Été,0.000003,257.67
3,2012-04,79898631,2012,Été,0.000003,239.70
4,2012-05,70076935,2012,Été,0.000003,210.23
...,...,...,...,...,...,...
149,2024-06,60758870,2024,Été,0.000009,546.83
150,2024-07,64371145,2024,Été,0.000009,579.34
151,2024-08,61545322,2024,Été,0.000009,553.91
152,2024-09,61819655,2024,Été,0.000009,556.38





---




**Pt2. Predection des couts energitiques**

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [107]:

# Séparer les variables indépendantes et la variable cible pour la consommation et le coût énergétique
X = monthly_consumption[['Année', 'Saison', 'Unité énergétique']]
y_consumption = monthly_consumption['Consommation électricité (MW)']
y_cost = monthly_consumption['Coût énergétique (DH)']


In [108]:
# Encoder la colonne "Saison" en valeurs numériques (One Hot Encoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('saison', OneHotEncoder(), ['Saison'])
    ], remainder='passthrough'
)







In [109]:
# Créer un modèle de régression linéaire avec pipeline
model_consumption = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model_cost = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [110]:
# Diviser les données en ensembles d'entraînement et de test (80% train, 20% test)
X_train, X_test, y_train_consumption, y_test_consumption = train_test_split(X, y_consumption, test_size=0.2, random_state=42)
X_train, X_test, y_train_cost, y_test_cost = train_test_split(X, y_cost, test_size=0.2, random_state=42)


In [111]:
# Entraîner les modèles sur l'ensemble d'entraînement
model_consumption.fit(X_train, y_train_consumption)
model_cost.fit(X_train, y_train_cost)

/usr/local/lib/python3.10/dist-packages/sklearn/compose/_column_transformer.py:1651: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('saison', OneHotEncoder(),
                                                  ['Saison'])])),
                ('regressor', LinearRegression())])

In [112]:
# Prédire sur l'ensemble de test
y_pred_consumption = model_consumption.predict(X_test)
y_pred_cost = model_cost.predict(X_test)

In [113]:
# Calculer les métriques de performance pour la consommation (RMSE et R2)
rmse_consumption = mean_squared_error(y_test_consumption, y_pred_consumption)
r2_consumption = r2_score(y_test_consumption, y_pred_consumption)

# Calculer les métriques de performance pour le coût énergétique (RMSE et R2)
rmse_cost = mean_squared_error(y_test_cost, y_pred_cost)
r2_cost = r2_score(y_test_cost, y_pred_cost)

# Afficher les résultats
print(f"RMSE pour la consommation (MW) sur l'ensemble de test : {rmse_consumption}")
print(f"R² pour la consommation (MW) sur l'ensemble de test : {r2_consumption}")
print(f"RMSE pour le coût énergétique (DH) sur l'ensemble de test : {rmse_cost}")
print(f"R² pour le coût énergétique (DH) sur l'ensemble de test : {r2_cost}")


RMSE pour la consommation (MW) sur l'ensemble de test : 58276897092247.53
R² pour la consommation (MW) sur l'ensemble de test : 0.6531441845119774
RMSE pour le coût énergétique (DH) sur l'ensemble de test : 1651.7609873744893
R² pour le coût énergétique (DH) sur l'ensemble de test : 0.9028344978005876


In [115]:

# Prédire pour de nouvelles données (exemple avec l'année 2025 et la saison 'Été')
new_data = pd.DataFrame({
    'Année': [2025],
    'Saison': ['Été'],
    'Unité énergétique': [0.000009]  # Exemple d'unité énergétique
})

# Prédictions
new_consumption = model_consumption.predict(new_data)
#new_cost = new_consumption*new_data['Unité énergétique']
new_cost = model_cost.predict(new_data)

print(f"Prédiction de la consommation pour 2025 (Été): {new_consumption[0]} MW")
print(f"Prédiction du coût énergétique pour 2025 (Été): {new_cost[0]} DH")

Prédiction de la consommation pour 2025 (Été): 66480939.75714111 MW
Prédiction du coût énergétique pour 2025 (Été): 622.4439959941219 DH


**Pt3. Sauvgarder mes models**

In [116]:
import joblib

# Sauvegarder le modèle de consommation
joblib.dump(model_consumption, '/content/model_consumption.pkl')

# Sauvegarder le modèle de coût énergétique
joblib.dump(model_cost, '/content/model_cost.pkl')


['/content/model_cost.pkl']

In [117]:
from google.colab import files

# Télécharger les fichiers sur votre machine locale
files.download('/content/model_consumption.pkl')
files.download('/content/model_cost.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>